# Spam Classifier

In [1]:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from common.utils import get_data_urls, fetch_data, get_labeled_files
from common.message import Message

In [2]:
#data_urls = get_data_urls()
#fetch_data(data_urls)

data_files, class_labels = get_labeled_files()

In [3]:
from sklearn.model_selection import train_test_split

train_files, test_files, y_train, y_test = train_test_split(data_files, class_labels,
                                                            test_size=0.2, random_state=44)
print(f"Training set size is\t {len(train_files)}")
print(f"Test set size is\t {len(test_files)}")

Training set size is	 7479
Test set size is	 1870


In [4]:
from common.message import Message
from common.message_transformer import MessageTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ("message_trf", MessageTransformer()),
    ("vectorizer", 
         ColumnTransformer([
            ("tdidf_body_vectorizer", TfidfVectorizer(max_features=5000), "tokens"),
            ("std_scaler", StandardScaler(), ["num_links", "cap_max"])
         ], remainder="passthrough")
    )
])

In [5]:
from mailparser import mailparser
from common.message import Message

labels = y_train
features = pipeline.fit_transform(train_files)

More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from unknown HELO mfrenchw2k mfrench42@62.254.163.42 with login by smtp.mail.vip.sc5.yahoo.com with SMTP; 13 Aug 2002 09:18:55 -0000
More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from unknown HELO mfrenchw2k mfrench42@62.254.163.42 with login by smtp.mail.vip.sc5.yahoo.com with SMTP; 13 Aug 2002 09:18:55 -0000
More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from chekitb5876.com 195.27.92.154 by asptown.co.kr 211.52.47.8 with Nmail V3.1 20010905 S for <jm@netnoteinc.com> from <jimmiester@hanmesoft.co.kr>; Sun, 02 Jun 2002 02:37:30 +0900
More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*

In [6]:
import joblib

joblib.dump(features, "features.pkl");

In [8]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

parameters = {
    "lambda": (0.0, 0.0001, 0.01),
    "alpha": (0.0, 0.0001, 0.01),
}
xgb_clf = xgb.XGBClassifier(booster="gblinear")
grid_search = GridSearchCV(xgb_clf, parameters, cv=5)
grid_search.fit(features, labels)

classifier = grid_search.best_estimator_
print("Best estimates:", grid_search.best_params_)

KeyboardInterrupt: 

In [9]:
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier().fit(features, labels)
dummy_train_accuracy = np.mean(cross_val_score(dummy_clf, features, labels, cv=5))
xgb_train_accuracy = np.mean(cross_val_score(classifier, features, labels, cv=5))
print("XGB Accuracy:\t", xgb_train_accuracy)
print("Dummy Accuracy\t", dummy_train_accuracy)

XGB Accuracy:	 0.9782063205336862
Dummy Accuracy	 0.6177306708636632


In [10]:
import joblib

joblib.dump(pipeline, "pipeline.pkl");
joblib.dump(classifier, "classifier.pkl");